# About this notebook

When I saw [Chris Deotte's][1] wonderful notebook in Feedback Prize competition,

 https://www.kaggle.com/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics 

I wondered if it could be classified using UMAP for image embedding.

Thank you for Chris Deotte! I respect you.

In this notebook, I showed a result, dogs and cats could be easily classified by using UMAP for image embedding. 

[1]:https://www.kaggle.com/cdeotte

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import PIL

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# 0. Import timm

In [ ]:
!pip install timm

In [ ]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

# 1.Loading train data

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train

In [ ]:
path = "../input/petfinder-pawpularity-score/train"

train["path"] = [os.path.join(path,s) + ".jpg" for s in train["Id"]]

In [ ]:
train.head(3)

## show 1 image

In [ ]:
img = cv2.imread(train["path"].iloc[0])
plt.imshow(img[:,:,::-1])

# 2. Making Image embedding

## 2.1 Dataset/Dataloader

For the basic writing code, I referred to the following notebook.


ref ) https://www.kaggle.com/titericz/imagenet-embeddings-plus-rapids-svr @GIBA. Thank you very much!

In [ ]:
class timmDataset:
    def __init__(self, path, modelcfg=None):
        
        self.path = path
        self.transform = create_transform(**modelcfg)
        
        
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, item):
        
        impath = self.path[item]
        img = Image.open(impath).convert('RGB')
        img = self.transform(img) # PIL image
        return img


In [ ]:
modelname = 'efficientnet_b0'

In [ ]:
model = timm.create_model(modelname, pretrained=True)

In [ ]:
model.to("cuda")
model.eval()

In [ ]:
model.default_cfg

In [ ]:
train_dataset = timmDataset(
    train["path"],
    modelcfg = resolve_data_config({},model=modelname)
)

In [ ]:
train_dataset[0]

In [ ]:
train_dataset[0].shape

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=256, num_workers= 2, shuffle=False)

In [ ]:
for a in train_dataloader:
    print(a)
    break

## 2.2 making image embedding with prediction

In [ ]:
from tqdm import tqdm

In [ ]:
allpreds = []

with torch.no_grad():
    for a in tqdm(train_dataloader):
        preds = model(a.to("cuda"))
        preds = preds.detach().cpu().numpy()
        allpreds.append(preds)
    

In [ ]:
len(allpreds)

In [ ]:
allpreds = np.concatenate(allpreds)
len(allpreds)

In [ ]:
preddf = pd.DataFrame(allpreds)

In [ ]:
preddf

# 3. UMAP
(ref : https://www.kaggle.com/cdeotte/rapids-umap-tfidf-kmeans-discovers-15-topics 
thank you for @chris deotte)


In [ ]:
import cudf, cuml, cupy
from cuml import UMAP


In [ ]:
umap = UMAP()
embed_2d = umap.fit_transform(preddf.values)
embed_2d = cupy.asnumpy( embed_2d )

In [ ]:
plt.scatter(embed_2d[:,0],embed_2d[:,1])

In [ ]:
from cuml import KMeans
kmeans = cuml.KMeans(n_clusters=2)
kmeans.fit(embed_2d)
train['cluster'] = kmeans.labels_

In [ ]:
plt.scatter(embed_2d[:,0],embed_2d[:,1],c = kmeans.labels_)

## maybe dog and cats. check now

In [ ]:
train.head(3)

## Cluster 0

In [ ]:
tmpdf = train[train["cluster"]==0]
plt.figure(figsize=(25,25))

for a in range(25):
    img = cv2.imread(tmpdf["path"].iloc[a])
    img = img[:,:,::-1]
    plt.subplot(5,5,a+1)
    plt.axis("off")
    plt.imshow(img)


### All dogs !!

## Cluster 1

In [ ]:
tmpdf = train[train["cluster"]==1]
plt.figure(figsize=(25,25))

for a in range(25):
    img = cv2.imread(tmpdf["path"].iloc[a])
    img = img[:,:,::-1]
    plt.subplot(5,5,a+1)
    plt.axis("off")
    plt.imshow(img)


## All cats !!

# 4. (Ref) t-SNE : no good case

In [ ]:
preddf

In [ ]:
from cuml.manifold import TSNE
tsne = TSNE(n_components=2)
X_embedded = tsne.fit_transform(preddf.values)

In [ ]:
plt.scatter(X_embedded[:,0],X_embedded[:,1])

## this is not good compared to UMAP (poor condition ?)

# 5.Find similar image knn

This is also referred from @chris deotte's notebook in Shoppee competition. Thank you so much !
https://www.kaggle.com/cdeotte/rapids-cuml-tfidfvectorizer-and-knn

In [ ]:
from cuml.neighbors import NearestNeighbors

In [ ]:
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(preddf.values)
distances, indices = model.kneighbors(preddf.values)

In [ ]:
distances

In [ ]:
indices

## Try to put out 5 similar images of no 0

In [ ]:
indices[0][:6]

In [ ]:
plt.figure(figsize=(20,5))
for num,a in enumerate(indices[0][:6]):
    img = cv2.imread(train["path"].iloc[a])
    img = img[:,:,::-1]
    
    title =train["Pawpularity"].iloc[a]
    plt.subplot(1,6,num+1)
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Pawpularity : {title}")


## it looks good.

# 6.Cosign Similarity : extract duplicate image

## 6.1 normalization

In [ ]:
preddf

### pandas → cudf

In [ ]:
predcudf = cudf.from_pandas(preddf)

## normalization each image embedding

In [ ]:
im_embed_norm = [s / cupy.linalg.norm(s) for s in predcudf.values]

## cudf → cupy

In [ ]:
im_embed_norm = cupy.array(im_embed_norm)

## cosine similality

In [ ]:
cos_sim = cupy.matmul( im_embed_norm, im_embed_norm.T).T

In [ ]:
cos_sim

## check histgram

In [ ]:
simdf = cudf.DataFrame(cos_sim)
simdf

## cudf → pandas

In [ ]:
simdf = simdf.to_pandas()

In [ ]:
plt.hist(np.concatenate(simdf.values))

## np.where(condition) put out the index satisfied with the condition

### you can change the threshold

In [ ]:
similarity_thres = 0.87

In [ ]:
similar_index = [cupy.where(s > similarity_thres)[0] for s in cos_sim]

In [ ]:
similar_index[:3]

## make combination id

In [ ]:
sim_combination = [train.iloc[cupy.asnumpy(s)].index.values for s in similar_index]

In [ ]:
sim_combination[:3]

## make judge whether duplicate or not

In [ ]:
duplicatejudge = [len(s) > 1 for s in sim_combination]

In [ ]:
train["combi"] = sim_combination
train["judge"] = duplicatejudge

In [ ]:
train["combi"] = train["combi"].astype("str")

## visualize duplicate image

In [ ]:
dupdf = train[train["judge"]]
dupdf = dupdf.sort_values("combi")

In [ ]:
dupdf.head()

In [ ]:
for a in range(int(len(dupdf)/2)):
    
    img = cv2.imread(dupdf["path"].iloc[2*a])
    img2 = cv2.imread(dupdf["path"].iloc[2*a+1])
    
    fig = plt.figure(figsize = (10,5))
    
    plt.subplot(1,2,1)
    plt.imshow(img[:,:,::-1])
    plt.axis("off")
    
    score = dupdf["Pawpularity"].iloc[2*a]
    plt.title(f"id{dupdf.index[2*a]}. Pawpularity {score}")
    
    
    plt.subplot(1,2,2)
    plt.imshow(img2[:,:,::-1])
    plt.axis("off")
    
    score = dupdf["Pawpularity"].iloc[2*a+1]
    plt.title(f"id{dupdf.index[2*a+1]}. Pawpularity {score}")
    
    del fig
    

In [ ]:
dupdf.to_csv("dupdf.csv",index=False)